In [1]:
import json
import pandas as pd

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, multilabel_confusion_matrix

In [2]:
df = pd.read_csv('./prediction_results.csv')

label_map = ['toxic','obscene','threat','insult','identity_attack']
hive_map = {
    "sexual":["obscene","insult"],
    "hate":["identity_attack"], 
    "violence":["threat"], 
    "bullying":["toxic"]
}
preds_openai=[]
preds_hive=[]
for i,row in df.iterrows():
    #Parse Hive
    if type(row.pred_hive) != str:
        pred_hive = []
    else:
        pred_hive = [item['class'] for item in json.loads(row.pred_hive.replace("'",'"')) if item['score'] > 0]
        pred_hive = [hive_map[pred] for pred in pred_hive if pred in hive_map]
        #flatten list of lists
        pred_hive = [item for sublist in pred_hive for item in sublist]
    preds_hive.append(pred_hive)

    # Parse OpenAI
    if row.pred_openai is None:
        pred_openai = []
    else:
        pred_openai=[]
        for item in row.pred_openai.split(','):
            if item.strip().isnumeric() and int(item) >= 1 and int(item) <= 5:
                pred_openai.append(label_map[int(item)-1])
            elif item.lower() in  label_map:
                pred_openai.append(item.lower())
            elif "no categories" in item.lower() or "none" in item.lower() or item=="0":
                item = None
            elif "." in item:
                pred_openai.append(label_map[int(item.split(".")[0])-1])
            else:
                print("Undefined prediction: ",item)

    preds_openai.append(pred_openai)


df['pred_openai'] = preds_openai             
df['pred_hive'] = preds_hive         
df['custom'] = df['custom'].apply(lambda x:json.loads(x.replace("'",'"')))                 
df['custom'] = df['custom'].apply(lambda x:[item.replace("identity_hate","identity_attack").replace("severe_toxic","toxic") for item in x])                 
df['custom'] = df['custom'].apply(lambda x:list(set(x)))                 
    
df['label'] = df.apply(lambda x: [
        label
        for label in label_map
        if int(x[label]) == 1
    ], axis=1)
df.head(3)    

,text,toxic,obscene,threat,insult,identity_attack,pred_openai,pred_hive,custom,pred_openai_binary,label
0,Trump est un bouffon traître.,1,0,0,1,0,[insult],[toxic],"[toxic, insult]",No,"[toxic, insult]"
1,Combien de temps encore notre pays peut-il per...,1,1,0,1,0,"[insult, identity_attack]",[],"[toxic, obscene]",No.,"[toxic, obscene, insult]"
2,LMAO. Mexicains blancs contre Mexicains bruns....,1,0,0,0,1,[insult],[],"[toxic, identity_attack]",No,"[toxic, identity_attack]"


# 1 .Multilabel Evaluation

In [3]:
mlb = MultiLabelBinarizer()
mlb.fit_transform([label_map])

array([[1, 1, 1, 1, 1]])

## 1.1. Hive

In [4]:
print(
    classification_report(
        mlb.transform(df.label.to_list()),
        mlb.transform(df.pred_hive.to_list()), 
        target_names=mlb.classes_,
        zero_division=0
        )
)

                 precision    recall  f1-score   support

identity_attack       0.77      0.48      0.59       139
         insult       0.24      0.07      0.10       287
        obscene       0.16      0.10      0.13       128
         threat       0.49      0.67      0.56       100
          toxic       0.89      0.27      0.42       396

      micro avg       0.54      0.26      0.35      1050
      macro avg       0.51      0.32      0.36      1050
   weighted avg       0.57      0.26      0.33      1050
    samples avg       0.19      0.16      0.16      1050



## 1.2. OpenAI

In [5]:
print(
    classification_report(
        mlb.transform(df.label.to_list()),
        mlb.transform(df.pred_openai.to_list()), 
        target_names=mlb.classes_,
        zero_division=0
        )
)

                 precision    recall  f1-score   support

identity_attack       0.22      0.53      0.31       139
         insult       0.37      0.83      0.51       287
        obscene       0.22      0.03      0.05       128
         threat       0.60      0.06      0.11       100
          toxic       0.56      0.02      0.04       396

      micro avg       0.32      0.32      0.32      1050
      macro avg       0.39      0.30      0.21      1050
   weighted avg       0.43      0.32      0.21      1050
    samples avg       0.21      0.20      0.19      1050



## 1.3. Custom Model

In [6]:
print(
    classification_report(
        mlb.transform(df.label.to_list()),
        mlb.transform(df.custom.to_list()), 
        target_names=mlb.classes_,
        zero_division=0
        )
)

                 precision    recall  f1-score   support

identity_attack       0.92      0.96      0.94       139
         insult       0.80      0.91      0.85       287
        obscene       0.90      0.89      0.89       128
         threat       0.99      0.91      0.95       100
          toxic       0.59      0.96      0.73       396

      micro avg       0.73      0.93      0.82      1050
      macro avg       0.84      0.93      0.87      1050
   weighted avg       0.77      0.93      0.83      1050
    samples avg       0.49      0.60      0.52      1050



# 2. Binary Evaluation

## 2.1. Hive

In [7]:
pred_binary = df.pred_hive.apply(lambda x: len(x) > 0).astype(int).to_list()
true_binary = df.label.apply(lambda x: len(x) > 0).astype(int).to_list()

print(
    classification_report(
        true_binary,
        pred_binary, 
        target_names=["Normal","Toxic"]
    )
)

              precision    recall  f1-score   support

      Normal       0.48      0.93      0.64       350
       Toxic       0.93      0.46      0.61       650

    accuracy                           0.62      1000
   macro avg       0.70      0.70      0.62      1000
weighted avg       0.77      0.62      0.62      1000



## 2.2. OpenAI

### Converting Multi-Label to Binary

In [8]:
pred_binary = df.pred_openai.apply(lambda x: len(x) > 0).astype(int).to_list()
true_binary = df.label.apply(lambda x: len(x) > 0).astype(int).to_list()

print(
    classification_report(
        true_binary,
        pred_binary, 
        target_names=["Normal","Toxic"]
    )
)

              precision    recall  f1-score   support

      Normal       0.55      0.54      0.54       350
       Toxic       0.75      0.76      0.76       650

    accuracy                           0.68      1000
   macro avg       0.65      0.65      0.65      1000
weighted avg       0.68      0.68      0.68      1000



### Asking for Binary Prompt

In [9]:
pred_binary = df.pred_openai_binary.apply(lambda x: "yes" in x.lower()).astype(int).to_list()
true_binary = df.label.apply(lambda x: len(x) > 0).astype(int).to_list()

print(
    classification_report(
        true_binary,
        pred_binary, 
        target_names=["Normal","Toxic"]
    )
)

              precision    recall  f1-score   support

      Normal       0.39      0.99      0.56       350
       Toxic       0.96      0.16      0.27       650

    accuracy                           0.45      1000
   macro avg       0.68      0.57      0.42      1000
weighted avg       0.76      0.45      0.37      1000



## 2.3. Custom Model

In [10]:
pred_binary = df.custom.apply(lambda x: len(x) > 0).astype(int).to_list()
true_binary = df.label.apply(lambda x: len(x) > 0).astype(int).to_list()

print(
    classification_report(
        true_binary,
        pred_binary, 
        target_names=["Normal","Toxic"]
    )
)

              precision    recall  f1-score   support

      Normal       0.92      0.86      0.89       350
       Toxic       0.93      0.96      0.94       650

    accuracy                           0.92      1000
   macro avg       0.92      0.91      0.92      1000
weighted avg       0.92      0.92      0.92      1000

